# Thanks to https://www.kaggle.com/siavrez/wavenet-keras and Sergey Bryansky.
# You can take a look at Sergey's kernel [here](https://www.kaggle.com/sggpls/shifted-rfc-pipeline) or [here](https://www.kaggle.com/sggpls/wavenet-with-shifted-rfc-proba). Also, Sergey's [data is here.](https://www.kaggle.com/sggpls/ion-shifted-rfc-proba)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install tensorflow_addons
import tensorflow as tf
from tensorflow.keras.layers import *
import pandas as pd
import numpy as np
import random
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import losses, models, optimizers
import tensorflow_addons as tfa
import gc

from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
# configurations and main hyperparammeters
EPOCHS = 180
NNBATCHSIZE = 16
GROUP_BATCH_SIZE = 4000
SEED = 321
LR = 0.001
SPLITS = 5



# Feature Engineering

In [0]:
# read data
def read_data():
#     train = pd.read_csv('/kaggle/input/data-without-drift/train_clean.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
#     train = pd.read_csv('/kaggle/input/clean-batch7/train_clean_batch7.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    train = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/train_clean_batch7_gause.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})    
    test  = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/test_clean.csv', dtype={'time': np.float32, 'signal': np.float32})
#     train = pd.read_csv('/kaggle/input/clean-kalman/train_clean_kalman.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
#     test  = pd.read_csv('/kaggle/input/clean-kalman/test_clean_kalman.csv', dtype={'time': np.float32, 'signal': np.float32})

    sub  = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/submission/sample_submission.csv', dtype={'time': np.float32})
    
    Y_train_proba = np.load("./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/Y_train_proba.npy")
    Y_test_proba = np.load("./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/Y_test_proba.npy")
       
    for i in range(11):
        train[f"proba_{i}"] = Y_train_proba[:, i]
        test[f"proba_{i}"] = Y_test_proba[:, i]

#     lgbm_train = pd.read_csv("/kaggle/input/lgbm-pred/train_sub.csv")
#     lgbm_test = pd.read_csv("/kaggle/input/lgbm-pred/test_sub.csv")
    
#     train['lgbm_pred'] = lgbm_train['open_channels']
#     test['lgbm_pred'] = lgbm_test['open_channels']

    # lowpass_train = pd.read_csv("./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/train_lowpass_ff_0.5012.csv")
    # lowpass_test = pd.read_csv("./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/test_lowpass_ff_0.5012.csv")
    
    # train['lowpass_ff_0.5012'] = lowpass_train['lowpass_ff_0.5012']
    # test['lowpass_ff_0.5012'] = lowpass_test['lowpass_ff_0.5012']

    return train, test, sub

# create batches of 4000 observations
def batching(df, batch_size):
    df['group'] = df.groupby(df.index//batch_size, sort=False)['signal'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
    return df

# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    train_input_mean = train.signal.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal - train_input_mean) / train_input_sigma
    test['signal'] = (test.signal - train_input_mean) / train_input_sigma
    return train, test

# get lead and lags features
def lag_with_pct_change(df, windows):
    for window in windows:    
        df['signal_shift_pos_' + str(window)] = df.groupby('group')['signal'].shift(window).fillna(0)
        df['signal_shift_neg_' + str(window)] = df.groupby('group')['signal'].shift(-1 * window).fillna(0)
    return df
  
def remove_batch7_spike(df):
    b_size = 500000
    
    df['signal_aranged'] = df['signal'].values
    data_7 = df[7*b_size:8*b_size]
    batch7_outlier_idx = data_7.query('signal_aranged >= 2.2 or signal_aranged <= -3.7').index
    df['signal_aranged'][batch7_outlier_idx] = 1
    
    return df



# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering(df, batch_size):
    # create batches
    df = batching(df, batch_size = batch_size)
    # create leads and lags (1, 2, 3 making them 6 features)
    df = lag_with_pct_change(df, [1, 2, 3])
    
#     df = remove_batch7_spike(df)
    

#     df = rolling
    # create signal ** 2 (this is the new feature)
#     df['signal_2'] = df['signal'] ** 2
    return df

# fillna with the mean and select features for training
def feature_selection(train, test):
    features = [col for col in train.columns if col not in ['index', 'group', 'open_channels', 'time']]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in features:
        feature_mean = pd.concat([train[feature], test[feature]], axis = 0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024 ** 2 # just added 
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024 ** 2
    percent = 100 * (start_mem - end_mem) / start_mem
    print('Mem. usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)'.format(start_mem, end_mem, percent))
    return df

# kalman!!!!

def read_kalman():
#     train = pd.read_csv('/kaggle/input/data-without-drift/train_clean.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
#     test  = pd.read_csv('/kaggle/input/data-without-drift/test_clean.csv', dtype={'time': np.float32, 'signal': np.float32})
#     train = pd.read_csv('/kaggle/input/clean-kalman/train_clean_kalman.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
#     train = pd.read_csv('/kaggle/input/clean-batch7/train_clean_kalman_batch7.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    train = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/train_clean_kalman_batch7_gause.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    test  = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/test_clean_kalman.csv', dtype={'time': np.float32, 'signal': np.float32})

    return train, test


# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering_k(df, batch_size):
    
#     df = remove_batch7_spike(df)

    return df

def connect_kalman(df, df_kalman):
#     df_kalman = remove_batch7_spike(df_kalman)
    df['kalman_sig'] = df_kalman['signal'].values
#     df['kalman_aranged'] = df_kalman['signal_aranged'].values
    
    return df

# original!!!!

def read_original():
#     train = pd.read_csv('/kaggle/input/data-without-drift/train_clean.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
#     test  = pd.read_csv('/kaggle/input/data-without-drift/test_clean.csv', dtype={'time': np.float32, 'signal': np.float32})
    train = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/train.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    test  = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/test.csv', dtype={'time': np.float32, 'signal': np.float32})

    return train, test

def connect_raw(df, df_raw):
    df['raw_sig'] = df_raw['signal'].values
    
    return df
    

In [5]:
# this function run our entire program

print('Reading Data Started...')
train, test, sample_submission = read_data()
train, test = normalize(train, test)
print('Reading and Normalizing Data Completed')

print('Creating Features')
print('Feature Engineering Started...')
train = run_feat_engineering(train, batch_size = GROUP_BATCH_SIZE)
test = run_feat_engineering(test, batch_size = GROUP_BATCH_SIZE)



# kalman
print('Reading Data Started...')
train_k, test_k = read_kalman()
train_k, test_k = normalize(train_k, test_k)
print('Reading and Normalizing Data Completed')

print('Creating Features')
print('Feature Engineering Started...')
train_k = run_feat_engineering_k(train_k, batch_size = GROUP_BATCH_SIZE)
test_k = run_feat_engineering_k(test_k, batch_size = GROUP_BATCH_SIZE)

train = connect_kalman(train, train_k)
test = connect_kalman(test, test_k)
# kalman end

# original
# print('Reading Data Started...')
# train_raw, test_raw = read_original()
# train_raw, test_raw = normalize(train_raw, test_raw)
# print('Reading and Normalizing Data Completed')
# train = connect_raw(train, train_raw)
# test = connect_raw(test, test_raw)
# original end

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
train.head()

Reading Data Started...
Reading and Normalizing Data Completed
Creating Features
Feature Engineering Started...
Reading Data Started...
Reading and Normalizing Data Completed
Creating Features
Feature Engineering Started...
Mem. usage decreased from 772.48 Mb to 205.04 Mb (73.5% reduction)
Mem. usage decreased from 301.36 Mb to 80.11 Mb (73.4% reduction)


,time,signal,open_channels,proba_0,proba_1,proba_2,proba_3,proba_4,proba_5,proba_6,proba_7,proba_8,proba_9,proba_10,group,signal_shift_pos_1,signal_shift_neg_1,signal_shift_pos_2,signal_shift_neg_2,signal_shift_pos_3,signal_shift_neg_3,kalman_sig
0,0.0001,-1.148438,0,0.966797,0.028336,0.004810,0.000114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,-1.187500,0.000000,-1.006836,0.000000,-1.302734,-1.152344
1,0.0002,-1.187500,0,0.996094,0.003466,0.000426,0.000063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1.148438,-1.006836,0.000000,-1.302734,0.000000,-1.307617,-1.187500
2,0.0003,-1.006836,0,0.976074,0.018982,0.004677,0.000021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1.187500,-1.302734,-1.148438,-1.307617,0.000000,-1.101562,-1.015625
3,0.0004,-1.302734,0,0.996094,0.003626,0.000326,0.000046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1.006836,-1.307617,-1.187500,-1.101562,-1.148438,-1.124023,-1.301758
4,0.0005,-1.307617,0,0.997559,0.002335,0.000158,0.000042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1.302734,-1.101562,-1.006836,-1.124023,-1.187500,-1.082031,-1.307617


Text(0.5, 1.0, 'Correlation')

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
# ---- set params ----
df = train
cmap = 'RdYlBu_r'
# --------------------
# Plot
fig, ax = plt.subplots(figsize=(12,10), dpi=100)
# plt.figure(figsize=(12,10), dpi= 80)
fig.patch.set_facecolor('white')
sns.heatmap(df.corr(),
            xticklabels=df.corr().columns,
            yticklabels=df.corr().columns,
            cmap=cmap,
            center=0,
            annot=False)
# Decorations
ax.set_title('Correlation', fontsize=22)

In [7]:
train, test, features = feature_selection(train, test)
print('Feature Engineering Completed...')
train.head()

Feature Engineering Completed...


,time,signal,open_channels,proba_0,proba_1,proba_2,proba_3,proba_4,proba_5,proba_6,proba_7,proba_8,proba_9,proba_10,group,signal_shift_pos_1,signal_shift_neg_1,signal_shift_pos_2,signal_shift_neg_2,signal_shift_pos_3,signal_shift_neg_3,kalman_sig
0,0.0001,-1.148438,0,0.966797,0.028336,0.004810,0.000114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000000,-1.187500,0.000000,-1.006836,0.000000,-1.302734,-1.152344
1,0.0002,-1.187500,0,0.996094,0.003466,0.000426,0.000063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1.148438,-1.006836,0.000000,-1.302734,0.000000,-1.307617,-1.187500
2,0.0003,-1.006836,0,0.976074,0.018982,0.004677,0.000021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1.187500,-1.302734,-1.148438,-1.307617,0.000000,-1.101562,-1.015625
3,0.0004,-1.302734,0,0.996094,0.003626,0.000326,0.000046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1.006836,-1.307617,-1.187500,-1.101562,-1.148438,-1.124023,-1.301758
4,0.0005,-1.307617,0,0.997559,0.002335,0.000158,0.000042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,-1.302734,-1.101562,-1.006836,-1.124023,-1.187500,-1.082031,-1.307617


# model

In [0]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [0]:
# model function (very important, you can try different arquitectures to get a better score. I believe that top public leaderboard is a 1D Conv + RNN style)
def Classifier(shape_):
    
    def cbr(x, out_layer, kernel, stride, dilation):
        x = Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x
    
    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = Multiply()([tanh_out, sigm_out])
            x = Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = Add()([res_x, x])
        return res_x
    
    inp = Input(shape = (shape_))
    x = cbr(inp, 64, 7, 1, 1)
    x = BatchNormalization()(x)
    # x = wave_block(x, 8, 3, 16)
    # x = BatchNormalization()(x)
    x = wave_block(x, 16, 3, 12)
    x = BatchNormalization()(x)
    x = wave_block(x, 32, 3, 8)
    x = BatchNormalization()(x)
    x = wave_block(x, 64, 3, 4)
    x = BatchNormalization()(x)
    x = wave_block(x, 128, 3, 1)
    x = BatchNormalization()(x)
    # x = wave_block(x, 64, 3, 1)
    x = cbr(x, 32, 7, 1, 1)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    out = Dense(11, activation = 'softmax', name = 'out')(x)
    
    model = models.Model(inputs = inp, outputs = out)
    
    opt = Adam(lr = LR)
    opt = tfa.optimizers.SWA(opt)
    model.compile(loss = losses.CategoricalCrossentropy(), optimizer = opt, metrics = ['accuracy'])
    return model

# function that decrease the learning as epochs increase (i also change this part of the code)
def lr_schedule(epoch):
    if epoch < 30:
        lr = LR
    elif epoch < 40:
        lr = LR / 3
    elif epoch < 50:
        lr = LR / 5
    elif epoch < 60:
        lr = LR / 7
    elif epoch < 70:
        lr = LR / 9
    elif epoch < 80:
        lr = LR / 11
    elif epoch < 90:
        lr = LR / 13
    else:
        lr = LR / 100
    return lr

# class to get macro f1 score. This is not entirely necessary but it's fun to check f1 score of each epoch (be carefull, if you use this function early stopping callback will not work)
class MacroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis = 2).reshape(-1)
        
    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis = 2).reshape(-1)
        score = f1_score(self.targets, pred, average = 'macro')
        print(f'F1 Macro Score: {score:.5f}')


In [0]:

# main function to perfrom groupkfold cross validation (we have 1000 vectores of 4000 rows and 8 features (columns)). Going to make 5 groups with this subgroups.
def run_cv_model_by_batch(train, test, splits, batch_col, feats, sample_submission, nn_epochs, nn_batch_size):
    
    seed_everything(SEED)
    K.clear_session()
    config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
    tf.compat.v1.keras.backend.set_session(sess)
    oof_ = np.zeros((len(train), 11)) # build out of folds matrix with 11 columns, they represent our target variables classes (from 0 to 10)
    preds_ = np.zeros((len(test), 11))
    target = ['open_channels']
    group = train['group']
    kf = GroupKFold(n_splits=5)
    splits = [x for x in kf.split(train, train[target], group)]

    new_splits = []
    for sp in splits:
        new_split = []
        new_split.append(np.unique(group[sp[0]]))
        new_split.append(np.unique(group[sp[1]]))
        new_split.append(sp[1])    
        new_splits.append(new_split)
    # pivot target columns to transform the net to a multiclass classification estructure (you can also leave it in 1 vector with sparsecategoricalcrossentropy loss function)
    tr = pd.concat([pd.get_dummies(train.open_channels), train[['group']]], axis=1)

    tr.columns = ['target_'+str(i) for i in range(11)] + ['group']
    target_cols = ['target_'+str(i) for i in range(11)]
    train_tr = np.array(list(tr.groupby('group').apply(lambda x: x[target_cols].values))).astype(np.float32)
    train = np.array(list(train.groupby('group').apply(lambda x: x[feats].values)))
    test = np.array(list(test.groupby('group').apply(lambda x: x[feats].values)))

    for n_fold, (tr_idx, val_idx, val_orig_idx) in enumerate(new_splits[0:], start=0):
        train_x, train_y = train[tr_idx], train_tr[tr_idx]
        valid_x, valid_y = train[val_idx], train_tr[val_idx]
        print(f'Our training dataset shape is {train_x.shape}')
        print(f'Our validation dataset shape is {valid_x.shape}')

        gc.collect()
        shape_ = (None, train_x.shape[2]) # input is going to be the number of feature we are using (dimension 2 of 0, 1, 2)
        model = Classifier(shape_)
        # using our lr_schedule function
        cb_lr_schedule = LearningRateScheduler(lr_schedule)
        model.fit(train_x,train_y,
                  epochs = nn_epochs,
                  callbacks = [cb_lr_schedule, MacroF1(model, valid_x, valid_y)], # adding custom evaluation metric for each epoch
                  batch_size = nn_batch_size,verbose = 2,
                  validation_data = (valid_x,valid_y))
        preds_f = model.predict(valid_x)
        f1_score_ = f1_score(np.argmax(valid_y, axis=2).reshape(-1),  np.argmax(preds_f, axis=2).reshape(-1), average = 'macro') # need to get the class with the biggest probability
        print(f'Training fold {n_fold + 1} completed. macro f1 score : {f1_score_ :1.5f}')
        preds_f = preds_f.reshape(-1, preds_f.shape[-1])
        oof_[val_orig_idx,:] += preds_f
        te_preds = model.predict(test)
        model.save("model-wavenet.h5")
        te_preds = te_preds.reshape(-1, te_preds.shape[-1])           
        preds_ += te_preds / SPLITS
    # calculate the oof macro f1_score
    f1_score_ = f1_score(np.argmax(train_tr, axis = 2).reshape(-1),  np.argmax(oof_, axis = 1), average = 'macro') # axis 2 for the 3 Dimension array and axis 1 for the 2 Domension Array (extracting the best class)
    print(f'Training completed. oof macro f1 score : {f1_score_:1.5f}')

    # oof saved
    train_copy = pd.read_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/train_clean_batch7_gause.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32}) 
    train_copy = train_copy.drop('signal', axis=1)
    train_copy['open_channels'] = np.argmax(oof_, axis = 1).astype(int)
    train_copy.to_csv('oof_wavenet.csv', index=False, float_format='%.4f')
    np.save('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/data/oof_wavenet.npy', oof_)
    np.save('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/submission/sub_wavenet.npy', preds_)

    sample_submission['open_channels'] = np.argmax(preds_, axis = 1).astype(int)
    sample_submission.to_csv('./drive/My Drive/Colab Notebooks/liverpool-ion-switching/submission/submission_wavenet_lowpass.csv', index=False, float_format='%.4f')
    


In [11]:

print(f'Training Wavenet model with {SPLITS} folds of GroupKFold Started...')
run_cv_model_by_batch(train, test, SPLITS, 'group', features, sample_submission, EPOCHS, NNBATCHSIZE)
print('Training completed...')
        

Training Wavenet model with 5 folds of GroupKFold Started...
Our training dataset shape is (1000, 4000, 19)
Our validation dataset shape is (250, 4000, 19)
Epoch 1/180
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
F1 Macro Score: 0.83297
63/63 - 39s - loss: 0.5547 - accuracy: 0.8538 - val_loss: 1.0086 - val_accuracy: 0.9235 - lr: 0.0010
Epoch 2/180
F1 Macro Score: 0.90442
63/63 - 27s - loss: 0.1957 - accuracy: 0.9581 - val_loss: 0.6463 - val_accuracy: 0.9518 - lr: 0.0010
Epoch 3/180
F1 Macro Score: 0.91970
63/63 - 27s - loss: 0.1639 - accuracy: 0.9620 - val_loss: 0.3326 - val_accuracy: 0.9610 - lr: 0.0010
Epoch 4/180
F1 Macro Score: 0.93336
63/63 - 27s - loss: 0.1357 - accuracy: 0.9660 - val_loss: 0.2010 - val_accuracy: 0.9649 - lr: 0.0010
Epoch 5/180
F1 Macro Score: 0.92982
63/63 - 27s - loss: 0.1281 - accuracy: 0.9665 - val_loss: 0.2163 - val_accuracy: 0.9647 - lr: 0.0010
Epoch 6/180
F1 Macro Score: 0.93659
63/63 - 27s - loss: 0.1224 - accuracy: 0.9